In [1]:
!pip install openai
!pip install langchain
!pip install Unidecode
!pip install unstructured
!pip install chromadb
!pip install tiktoken
import IPython
IPython.display.clear_output()

In [2]:
from langchain.llms.openai import OpenAI
from langchain.example_generator import generate_example
from langchain.prompts import PromptTemplate

from pprint import pprint
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup
import unidecode
import re
import csv


In [3]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.document_loaders import BSHTMLLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain, VectorDBQAWithSourcesChain
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains.llm import LLMChain
from langchain.chains.chat_vector_db.prompts import CONDENSE_QUESTION_PROMPT, QA_PROMPT
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts.prompt import PromptTemplate
from langchain.chat_models import ChatOpenAI

In [4]:
import os
os.environ["OPENAI_API_KEY"] = "<your_openai_apikey>"

In [5]:
def text_document_loader(dir_input):
    loader = DirectoryLoader(dir_input, glob="*", loader_cls=TextLoader)
    return loader

In [6]:
def unstruct_html_text_loader(dir_input):
    loader = DirectoryLoader(dir_input, glob="*", loader_cls=BSHTMLLoader)
    return loader

In [7]:
dir_input = "./dataset/knowledge/"
bhtml_loader = unstruct_html_text_loader(dir_input)

all_loader= [bhtml_loader]

In [8]:
ir_index = VectorstoreIndexCreator(vectorstore_kwargs={"persist_directory": "db"}).from_loaders(all_loader)

Using embedded DuckDB with persistence: data will be stored in: db


In [9]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT)
doc_chain = load_qa_with_sources_chain(llm, chain_type="stuff", verbose=True)

In [10]:
chain = ConversationalRetrievalChain(
    retriever=ir_index.vectorstore.as_retriever(),
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
)

In [11]:
def get_answer(query, chain, chat_history):
    result = None
    if chain is not None:
        result = chain({"question": query, "chat_history": chat_history})
    return result

In [12]:
chat_history = []
q = "what is my 401k limit?"
res = get_answer(q, chain, chat_history)
res



> Entering new LLMChain chain...
Prompt after formatting:
Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer.

QUESTION: Which state/country's law governs the interpretation of the contract?
Content: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights.
Source: 28-pl
Content: No Waiver. Failure or delay in exercising any right or remedy under this Agreement shall not constitute a waiver of such (or any other)  right or remedy.

11.7 Severability. The invalidity, illegality or unenforceability of any term (o


> Finished chain.


{'question': 'what is my 401k limit?',
 'chat_history': [],
 'answer': "The 401k limit depends on the type of contribution. For pre-tax contributions, the limit is $22,500 in 2023. For Roth Basic contributions, the limit is also $22,500 in 2023. The annual limit for NVIDIA's matching contributions is $9,000. After-tax contributions have a limit of $34,500 in addition to any pre-tax or Roth Basic contributions. \nSOURCES: dataset/knowledge/401K Retirement Plan & Corporate Matching _ NVIDIA Benefits.html"}

# Example of custom prompt

In [13]:
cust_template = """Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer.
QUESTION: Which state/country's law governs the interpretation of the contract?
=========
Content: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights.
Source: 28-pl
Content: No Waiver. Failure or delay in exercising any right or remedy under this Agreement shall not constitute a waiver of such (or any other)  right or remedy.\n\n11.7 Severability. The invalidity, illegality or unenforceability of any term (or part of a term) of this Agreement shall not affect the continuation  in force of the remainder of the term (if any) and this Agreement.\n\n11.8 No Agency. Except as expressly stated otherwise, nothing in this Agreement shall create an agency, partnership or joint venture of any  kind between the parties.\n\n11.9 No Third-Party Beneficiaries.
Source: 30-pl
Content: (b) if Google believes, in good faith, that the Distributor has violated or caused Google to violate any Anti-Bribery Laws (as  defined in Clause 8.5) or that such a violation is reasonably likely to occur,
Source: 4-pl
=========
FINAL ANSWER: This Agreement is governed by English law.
SOURCES: 28-pl
QUESTION: What did the president say about Michael Jackson?
=========
Content: Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. \n\nGroups of citizens blocking tanks with their bodies. Everyone from students to retirees teachers turned soldiers defending their homeland.
Source: 0-pl
Content: And we won’t stop. \n\nWe have lost so much to COVID-19. Time with one another. And worst of all, so much loss of life. \n\nLet’s use this moment to reset. Let’s stop looking at COVID-19 as a partisan dividing line and see it for what it is: A God-awful disease.  \n\nLet’s stop seeing each other as enemies, and start seeing each other for who we really are: Fellow Americans.  \n\nWe can’t change how divided we’ve been. But we can change how we move forward—on COVID-19 and other issues we must face together. \n\nI recently visited the New York City Police Department days after the funerals of Officer Wilbert Mora and his partner, Officer Jason Rivera. \n\nThey were responding to a 9-1-1 call when a man shot and killed them with a stolen gun. \n\nOfficer Mora was 27 years old. \n\nOfficer Rivera was 22. \n\nBoth Dominican Americans who’d grown up on the same streets they later chose to patrol as police officers. \n\nI spoke with their families and told them that we are forever in debt for their sacrifice, and we will carry on their mission to restore the trust and safety every community deserves.
Source: 24-pl
Content: And a proud Ukrainian people, who have known 30 years  of independence, have repeatedly shown that they will not tolerate anyone who tries to take their country backwards.  \n\nTo all Americans, I will be honest with you, as I’ve always promised. A Russian dictator, invading a foreign country, has costs around the world. \n\nAnd I’m taking robust action to make sure the pain of our sanctions  is targeted at Russia’s economy. And I will use every tool at our disposal to protect American businesses and consumers. \n\nTonight, I can announce that the United States has worked with 30 other countries to release 60 Million barrels of oil from reserves around the world.  \n\nAmerica will lead that effort, releasing 30 Million barrels from our own Strategic Petroleum Reserve. And we stand ready to do more if necessary, unified with our allies.  \n\nThese steps will help blunt gas prices here at home. And I know the news about what’s happening can seem alarming. \n\nBut I want you to know that we are going to be okay.
Source: 5-pl
Content: More support for patients and families. \n\nTo get there, I call on Congress to fund ARPA-H, the Advanced Research Projects Agency for Health. \n\nIt’s based on DARPA—the Defense Department project that led to the Internet, GPS, and so much more.  \n\nARPA-H will have a singular purpose—to drive breakthroughs in cancer, Alzheimer’s, diabetes, and more. \n\nA unity agenda for the nation. \n\nWe can do this. \n\nMy fellow Americans—tonight , we have gathered in a sacred space—the citadel of our democracy. \n\nIn this Capitol, generation after generation, Americans have debated great questions amid great strife, and have done great things. \n\nWe have fought for freedom, expanded liberty, defeated totalitarianism and terror. \n\nAnd built the strongest, freest, and most prosperous nation the world has ever known. \n\nNow is the hour. \n\nOur moment of responsibility. \n\nOur test of resolve and conscience, of history itself. \n\nIt is in this moment that our character is formed. Our purpose is found. Our future is forged. \n\nWell I know this nation.
Source: 34-pl
=========
FINAL ANSWER: The president did not mention Michael Jackson.
SOURCES:
QUESTION: {question}
=========
{summaries}
=========
FINAL ANSWER:"""
CUST_PROMPT = PromptTemplate(template=cust_template, input_variables=["summaries", "question"])

In [14]:
llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo")
question_generator = LLMChain(llm=llm, prompt=CUST_PROMPT)
doc_chain = load_qa_with_sources_chain(llm, chain_type="stuff", verbose=True, prompt=CUST_PROMPT )

chain = ConversationalRetrievalChain(
    retriever=ir_index.vectorstore.as_retriever(),
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
)

chat_history = []
q = "what is my 401k limit?"
res = get_answer(q, chain, chat_history)
res



> Entering new LLMChain chain...
Prompt after formatting:
Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer.
QUESTION: Which state/country's law governs the interpretation of the contract?
Content: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights.
Source: 28-pl
Content: No Waiver. Failure or delay in exercising any right or remedy under this Agreement shall not constitute a waiver of such (or any other)  right or remedy.

11.7 Severability. The invalidity, illegality or unenforceability of any term (or

/Users/aadityaramsh/anaconda3/envs/kratos/lib/python3.9/site-packages/langchain/llms/openai.py:169: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/aadityaramsh/anaconda3/envs/kratos/lib/python3.9/site-packages/langchain/llms/openai.py:608: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(



> Finished chain.


{'question': 'what is my 401k limit?',
 'chat_history': [],
 'answer': "The 401k limit depends on the type of contribution. For pre-tax contributions, the limit is $22,500 in 2023. For Roth Basic contributions, the limit is also $22,500 in 2023. The annual limit for NVIDIA's matching contributions is $9,000. After-tax contributions have a limit of $34,500 in addition to any pre-tax or Roth Basic contributions. \nSOURCES: dataset/knowledge/401K Retirement Plan & Corporate Matching _ NVIDIA Benefits.html"}

# Check top retrieved contents

In [15]:
res = ir_index.vectorstore.similarity_search_with_score(q, k=4)
res.sort(key=lambda x: x[1], reverse=True)
res

[(Document(page_content='Roth Basic. The money you contribute is deducted after federal and state taxes are withheld in applicable states. As a result, you may not be required to pay taxes on any of the earnings associated with your contributions in retirement as long as your distribution qualifies. Be sure to work with a tax advisor to ensure you understand the rules that qualify a Roth Basic distribution. Be advised that you do pay ordinary income tax on NVIDIA’s matching contributions when you withdraw your money in retirement. In 2023, the annual Roth limit is $22,500. (The IRS’ limit of $22,500 is a combined limit between Pre-Tax and Roth Basic contributions).', metadata={'source': 'dataset/knowledge/401K Retirement Plan & Corporate Matching _ NVIDIA Benefits.html', 'title': '401K Retirement Plan & Corporate Matching | NVIDIA Benefits'}),
  0.3810988962650299),
 (Document(page_content='After-tax. The money you save is deducted after taxes and other deductions are withheld. You can